In [285]:
import pandas as pd 
import os
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

data = pd.read_csv('/Users/natasha246/Documents/questions_list_users_sorted.csv', delimiter = ',')
df = pd.DataFrame(data)

In [286]:
ind = 0
questions = []
y = []
while ind!=500: 
    questions.append(data.iloc[ind, 4])
    y.append(data.iloc[ind,9])
    ind+=1    
y = np.array(y)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(questions)

In [287]:
import numpy as np
import csv
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from collections import defaultdict
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

In [288]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [289]:
y = []
ind = 0 
while ind!= 500:
    y.append(data.iloc[ind,9])
    ind+=1

In [290]:
lol_users = defaultdict(list)

for line in csv.DictReader(open('/Users/natasha246/Documents/questions_list_users_sorted.csv')):
    lol_users[line['User']] += tokenize(line['Text'])   
text_data = lol_users.values()

dictionary = Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
numtop = 10
model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=numtop)


lol_ = []
tm = []
users = []
for k, v in lol_users.items():
    bow = dictionary.doc2bow(v)
    doc_topics, word_topics, phi_values = model.get_document_topics(bow, per_word_topics=True)
    #print(k, doc_topics) 
    users.append(k)
    arr = [0] * numtop
    for tuple in doc_topics:
        index,prob = tuple
        arr[index] = prob
    lol_.append(arr)    

In [291]:
tm = []
counter = 0
ind = 0
for user in users:
    count = 0
    for line in csv.DictReader(open('/Users/natasha246/Documents/questions_list_users_sorted.csv')):
        if user == line['User']:
            count+=1
    i = 0
    while i!=count:
        tm.append(lol_[ind])
        i+=1
    ind +=1   

In [292]:
matrix = np.hstack((X.toarray(), tm))

In [293]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

from sklearn.model_selection import KFold 
from sklearn import metrics


kf = KFold(n_splits = 10, shuffle = True)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = matrix[train_index], matrix[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_train)
    predict = model.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, predict)) 

TRAIN: [  0   1   3   4   6   7   8   9  10  11  12  13  15  16  17  18  19  20
  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  38  39
  40  41  42  43  44  45  46  47  49  50  51  52  53  54  55  56  58  59
  60  61  62  63  64  65  66  68  69  70  71  72  73  74  75  76  77  78
  79  80  81  82  83  87  88  89  90  91  92  93  94  95  96  97  98 100
 101 102 103 104 105 107 108 109 110 111 113 114 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 136 137 138 139
 141 142 143 144 146 147 148 149 151 152 153 154 155 156 157 158 159 160
 161 163 164 165 166 168 169 170 171 173 174 175 176 177 178 179 180 181
 182 183 184 186 187 188 189 191 192 193 194 195 196 197 198 199 200 201
 202 203 204 205 206 207 208 210 211 212 215 216 217 218 220 221 222 223
 224 225 226 227 228 230 232 233 234 235 236 237 240 241 242 243 244 245
 246 247 248 249 250 251 252 253 254 255 256 257 258 259 261 262 263 264
 265 266 267 268 270 271 272 273 276 277 278

TypeError: only integer scalar arrays can be converted to a scalar index